In [4]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
// import pyspark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._
// import pyspark._

In [ ]:

val spark = SparkSession.builder.master("local")
            .appName("Task3_3")
//             .config("spark.driver.bindAddress", "localhost")
//             .config("spark.ui.port", "4040")
            .getOrCreate()

val schema = new StructType()
    .add("id", IntegerType) // id -  уникальный идентификатор посетителя сайта.
    .add("timestamp", IntegerType)// timestamp – дата и время события в формате unix timestamp.
    .add("type", StringType)// type – тип события, значение из списка 
    .add("page_id", IntegerType) // page_id – id текущей страницы. 
    .add("tag", StringType) // tag – каждая страница с новостью размечается редакцией специальными тегами
    .add("sign", BooleanType) // sign – наличие у пользователя личного кабинета.

val data = Seq(
    Row(1, 1667657000, "move", 1, "Sport", true),
    Row(2, 1667248464, "scroll", 1, "Sport", false),
    Row(3, 1667690325, "visit", 1, "medicine", true),
    Row(4, 1667687407, "click", 1, "Sport", false),
    Row(1, 1664237426, "move", 2, "Sport", true),
    Row(6, 1667626246, "scroll", 1, "Sport", true),
    Row(1, 1667607829, "click", 3, "politics", true),
    Row(7, 1667623826, "visit", 1, "Sport", false),
    Row(1, 1667027425, "click", 4, "politics", true),
    Row(8, 1667648928, "click", 1, "medicine", true),
    Row(3, 1667627926, "click", 3, "medicine", true),
    Row(1, 1667527420, "click", 5, "politics", true),
    Row(8, 1664627436, "move", 2, "medicine", true),
    Row(9, 1667685526, "click", 1, "politics", true),
    Row(2, 1667677420, "scroll", 1, "politics", false),
    Row(4, 1667225466, "click", 3, "Sport", false),
    Row(2, 1667727928, "click", 2, "Sport", false),
)

// Создайте датафрейм с описанной выше схемой данных.
// Наполните датафрейм данными
val df = spark.createDataFrame(spark.sparkContext.parallelize(data), schema)        
df.show()

// Вывести топ-5 самых активных посетителей сайта
val df_count = df.groupBy("id")
    .agg(count("id").as("cnt_id"))
    .sort(desc("cnt_id"))
    .show(5, truncate=false)

// Посчитать процент посетителей, у которых есть ЛК
var per_cent_df_with_lk = df.filter(df("sign")===true).select(countDistinct(df("id"))) //.show(false) 
// var per_cent_df_without_lk = df.filter(df("sign")===false).select(countDistinct(df("id"))).show(false)
// val count_id = df.select(df("id")).distinct().count().show(false)
// var count_id = (per_cent_df_with_lk * 100 / df.select(countDistinct(df("id")))).show(false)

// Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице
val df5 = df.filter(df("type")==="click").groupBy("page_id")
    .agg(count("type").as("cnt_type"))
    .sort(desc("cnt_type"))
    .show(5, truncate=false)

// Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)
var df_cl = df.withColumn("time",
                          when(col("id") === 1, "0-4")
                         .when(col("id") === 2, "4-8")
                         .when(col("id") === 3, "8-12")
                         .when(col("id") === 4, "0-4")
                         .when(col("id") === 5, "0-4")                         
                         .when(col("id") === 6, "8-12")
                         .when(col("id") === 7, "8-12")
                         .when(col("id") === 8, "0-4")
                         .when(col("id") === 9, "4-8")
                         .when(col("id") === 1, "4-8")
                         .when(col("id") === 2, "0-4")
                         .when(col("id") === 3, "8-12")
                         .when(col("id") === 4, "8-12")
                         .when(col("id") === 1, "0-4")
                         .cast(StringType))
df_cl.show()

// Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.
val df_time = df_cl.groupBy("time")
    .agg(count("time").as("cnt_time"))
    .sort(desc("cnt_time"))
    .select(first("time"))
    .show(false)

// Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта
val schema2 = new StructType()
    .add("id", IntegerType) // уникальный идентификатор личного кабинета
    .add("user_id", IntegerType)// tуникальный идентификатор посетителя
    .add("fio", StringType)// ФИО посетителя
    .add("date_cteate", StringType) // Дата создания ЛК

val data2 = Seq(
    Row(1, 1, "Orlov Oleg Olegovich", "2021-11-11"),
    Row(3, 3, "Ivanov Ivan Inanovich", "2022-10-01"),
    Row(6, 6, "Petrov Petr Petrovich", "2021-01-15"),
    Row(8, 0, "Maslov Maksim Maksimovich", "2022-03-03"),
    Row(9, 9, "Kotov Karl Karlovich", "2022-05-10")
)

val df2 = spark.createDataFrame(spark.sparkContext.parallelize(data2), schema2)        
df2.show()

// Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.
df.join(df2, df("id") === df2("user_id"), "inner")
    .filter(df("tag") === "Sport").select(df2("fio")).distinct().show(false)



+---+----------+------+-------+--------+-----+
| id| timestamp|  type|page_id|     tag| sign|
+---+----------+------+-------+--------+-----+
|  1|1667657000|  move|      1|   Sport| true|
|  2|1667248464|scroll|      1|   Sport|false|
|  3|1667690325| visit|      1|medicine| true|
|  4|1667687407| click|      1|   Sport|false|
|  1|1664237426|  move|      2|   Sport| true|
|  6|1667626246|scroll|      1|   Sport| true|
|  1|1667607829| click|      3|politics| true|
|  7|1667623826| visit|      1|   Sport|false|
|  1|1667027425| click|      4|politics| true|
|  8|1667648928| click|      1|medicine| true|
|  3|1667627926| click|      3|medicine| true|
|  1|1667527420| click|      5|politics| true|
|  8|1664627436|  move|      2|medicine| true|
|  9|1667685526| click|      1|politics| true|
|  2|1667677420|scroll|      1|politics|false|
|  4|1667225466| click|      3|   Sport|false|
|  2|1667727928| click|      2|   Sport|false|
+---+----------+------+-------+--------+-----+

